# Collecting Billboard Data from Wikipedia
#### 11/01/2023
In this notebook, I web scrape Wikipedia for Billboard data, in order to obtain a "ground truth" which I can use to compare with results from another source.

Wikipedia has many different articles for the Billboard Year-End Hot 100 singles of many years, from 1946 to 2022. Each of these articles is titled in the format "Billboard Year-End Hot 100 singles of `year`", which makes it really easy to search for each one. Additionally, each page features a large table containing the song title, artist, and ranking, which makes data very easy to collect.

## Testing out web scraping for the year 2000
Before trying to collect song data for all 23 years at once, let's practice doing this for one year (2000).

In [1]:
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
# Every URL will have this format
BASE_URL = "https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_"

In [3]:
year = 2000
URL = BASE_URL + str(year)

In [4]:
tables = pd.read_html(URL)
billboard_data = tables[0]
billboard_data['Year'] = year
billboard_data

,No.,Title,Artist(s),Year
0,1,"""Breathe""",Faith Hill,2000
1,2,"""Smooth""",Santana featuring Rob Thomas,2000
2,3,"""Maria Maria""",Santana featuring The Product G&B,2000
3,4,"""I Wanna Know""",Joe,2000
4,5,"""Everything You Want""",Vertical Horizon,2000
...,...,...,...,...
95,96,"""I Don't Wanna""",Aaliyah,2000
96,97,"""Independent Women Part I""",Destiny's Child,2000
97,98,"""Gotta Tell You""",Samantha Mumba,2000
98,99,"""Waiting for Tonight""",Jennifer Lopez,2000


Perfect – this strategy collected all 100 songs for the year 2000. Now let's perform this on all 23 years:

## Gathering data for all years, 2000-2023

In [5]:
years = list(range(2000, 2023))
all_data = []

for year in tqdm(years):
    URL = BASE_URL + str(year)
    tables = pd.read_html(URL)
    
    # Extract the first table and add the year column
    billboard_data = tables[0]
    billboard_data['Year'] = year
    
    # Append the data to our list
    all_data.append(billboard_data)

# Concatenate all the data into one dataframe
combined_data = pd.concat(all_data, ignore_index=True)
combined_data

100%|███████████████████████████████████████████| 23/23 [00:06<00:00,  3.45it/s]


,No.,Title,Artist(s),Year,№,0,1
0,1.0,"""Breathe""",Faith Hill,2000,NaN,NaN,NaN
1,2.0,"""Smooth""",Santana featuring Rob Thomas,2000,NaN,NaN,NaN
2,3.0,"""Maria Maria""",Santana featuring The Product G&B,2000,NaN,NaN,NaN
3,4.0,"""I Wanna Know""",Joe,2000,NaN,NaN,NaN
4,5.0,"""Everything You Want""",Vertical Horizon,2000,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2097,96.0,"""Flower Shops""",Ernest featuring Morgan Wallen,2022,NaN,NaN,NaN
2098,97.0,"""To the Moon""",Jnr Choi and Sam Tompkins,2022,NaN,NaN,NaN
2099,98.0,"""Unholy""",Sam Smith and Kim Petras,2022,NaN,NaN,NaN
2100,99.0,"""One Mississippi""",Kane Brown,2022,NaN,NaN,NaN


Strangely, it looks like we only have 2102 rows, when we were expecting 2300. Time to do some troubleshooting:

### Troubleshooting

Each year should have 100 songs, so to see which years are missing data, let's group the combined dataframe by year.

In [6]:
combined_data.groupby('Year').size()

Year
2000    100
2001    100
2002    100
2003    100
2004    100
2005    100
2006    100
2007    100
2008    100
2009    100
2010    100
2011    100
2012      1
2013      1
2014    100
2015    100
2016    100
2017    100
2018    100
2019    100
2020    100
2021    100
2022    100
dtype: int64

Interesting – 2012 and 2013 only have one row each. Let's see what they say:

In [7]:
combined_data[combined_data['Year'].isin([2012, 2013])]

,No.,Title,Artist(s),Year,№,0,1
1200,NaN,NaN,NaN,2012,NaN,NaN,This article relies excessively on references ...
1201,NaN,NaN,NaN,2013,NaN,NaN,This article relies excessively on references ...


After checking the wiki pages for 2012 and 2013, I found the problem. The issue is that the pages for these years both have a notice at the top, saying "This article relies excessively on references from primary sources..." The html reader must have interpreted this as the first table in the article, missing the main table that we were trying to read.

Instead of figuring out how to insert the 2012 and 2013 data into the currently existing data, let's just re-run the code we used earlier, but with a specification for the years 2012 and 2013:

### Re-gathering data for all years, with specifications for 2012 and 2013

In [8]:
all_data = []

for year in tqdm(years):
    url = base_url + str(year)
    tables = pd.read_html(url)
    
    # For the special cases of 2012 and 2013, extract the second table
    if year in [2012, 2013]:
        billboard_data = tables[1]
    else:
        billboard_data = tables[0]
    
    # Add the year column
    billboard_data['Year'] = year
    
    # Append the data to our list
    all_data.append(billboard_data)

# Concatenate all the DataFrames into one
combined_data = pd.concat(all_data, ignore_index=True)

# Display the first few rows to check
combined_data


100%|███████████████████████████████████████████| 23/23 [00:06<00:00,  3.37it/s]


,No.,Title,Artist(s),Year,№
0,1.0,"""Breathe""",Faith Hill,2000,NaN
1,2.0,"""Smooth""",Santana featuring Rob Thomas,2000,NaN
2,3.0,"""Maria Maria""",Santana featuring The Product G&B,2000,NaN
3,4.0,"""I Wanna Know""",Joe,2000,NaN
4,5.0,"""Everything You Want""",Vertical Horizon,2000,NaN
...,...,...,...,...,...
2295,96.0,"""Flower Shops""",Ernest featuring Morgan Wallen,2022,NaN
2296,97.0,"""To the Moon""",Jnr Choi and Sam Tompkins,2022,NaN
2297,98.0,"""Unholy""",Sam Smith and Kim Petras,2022,NaN
2298,99.0,"""One Mississippi""",Kane Brown,2022,NaN


Perfect – we now have 2300 rows as expected.

## Brief pre-processing

I notice from looking at the table that there's an extra column titled "№". It seems like different tables used different terms to denote ranking:

In [9]:
combined_data[combined_data['No.'].isnull()]

,No.,Title,Artist(s),Year,№
700,NaN,"""Irreplaceable""",Beyoncé,2007,1.0
701,NaN,"""Umbrella""",Rihanna featuring Jay-Z,2007,2.0
702,NaN,"""The Sweet Escape""",Gwen Stefani featuring Akon,2007,3.0
703,NaN,"""Big Girls Don't Cry""",Fergie,2007,4.0
704,NaN,"""Buy U a Drank (Shawty Snappin')""",T-Pain featuring Yung Joc,2007,5.0
...,...,...,...,...,...
895,NaN,"""The Way I Are""",Timbaland featuring Keri Hilson,2008,96.0
896,NaN,"""Addicted""",Saving Abel,2008,97.0
897,NaN,"""Into the Night""",Santana featuring Chad Kroeger,2008,98.0
898,NaN,"""Heaven Sent""",Keyshia Cole,2008,99.0


Instead of choosing to use one of the existing rows, I'll just make the ranking column from scratch. It's simply the remainder of the index divided by 100, plus one. Next, I'll remove the ugly quotes around the strings in the song titles, and remove any leftover unnecessary columns:

In [10]:
# Add ranking column
combined_data['Ranking'] = combined_data.index % 100 + 1

# Remove quotes around the titles
combined_data['Title'] = combined_data['Title'].apply(lambda s: s.strip('\"'))

# Remove No. columns
combined_data = combined_data.dropna(axis=1)

combined_data

,Title,Artist(s),Year,Ranking
0,Breathe,Faith Hill,2000,1
1,Smooth,Santana featuring Rob Thomas,2000,2
2,Maria Maria,Santana featuring The Product G&B,2000,3
3,I Wanna Know,Joe,2000,4
4,Everything You Want,Vertical Horizon,2000,5
...,...,...,...,...
2295,Flower Shops,Ernest featuring Morgan Wallen,2022,96
2296,To the Moon,Jnr Choi and Sam Tompkins,2022,97
2297,Unholy,Sam Smith and Kim Petras,2022,98
2298,One Mississippi,Kane Brown,2022,99


This is perfect – let's save it to a csv file for later use:

In [11]:
combined_data.to_csv('wiki_songs.csv', index=False)

## Next steps

Now that we have our "ground truth" data, let's see if our Spotify data matches up with it.